In [1]:
import hail as hl

Loading BokehJS ...

In [2]:
hl.init()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 45379
SPARKMONITOR_LISTENER: Application Started: application_1743531758511_0001 ...Start Time: 1743532661945


Running on Apache Spark version 3.5.0
SparkUI available at http://tk-m.c.nnf-karczewski.internal:37543
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.133-4c60fddb171a
LOGGING: writing to /home/hail/hail-20250401-1837-0.2.133-4c60fddb171a.log


# Helper

In [25]:
rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover('gs://hail-common/references/grch37_to_grch38.over.chain.gz', rg38)  

def liftover_37_38(ht):
    ht = ht.annotate(
        new_locus=hl.liftover(ht.locus, 'GRCh38', include_strand=True),
        old_locus=ht.locus
    )
    ht = ht.filter(hl.is_defined(ht.new_locus) & ~ht.new_locus.is_negative_strand)  
    ht = ht.key_by()
    ht = ht.annotate(locus=ht.new_locus.result)
    ht = ht.drop('new_locus', 'old_locus')
    return ht

# Munge Denovos

## DD

In [4]:
# already wrote the control files
dd_no_controls_1 = hl.read_table('gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/dd-lof-v1-no-controls.ht')

In [5]:
dd_no_controls_1.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'is_pos_dd': bool 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


## ASD

In [14]:
#ht_asd = hl.import_table('gs://genetics-gym/raw_data/Nat_Gen_published_autosomal_and_updated_XY_de_novo_calls_2024-05-13_no_cohort.txt')


2025-04-01 18:49:50.078 Hail: INFO: Loading <StructExpression of type struct{Variant: str, Chrom: str, Sample: str, Role: str, Sex: str, DDID: str, Father: str, Mother: str, Gene: str, Gene_ID: str, Transcript_ID: str, Consequence: str, Simplified_csq: str, HGVSc: str, HGVSp: str, loftee: str, loftee_flags: str, LOEUF: str, LOEUF_bin: str, isIndel: str, isCoding: str, isPTV: str, isMis: str, isSyn: str, MPC: str, am_pathogenicity: str, MPC_v2: str, MPC_v2_Outlier: str, XNonPAR: str, YNonPAR: str, isHemi: str, F_isHemi: str, nNonRef: str, Filters: str, VQSLOD: str, Call_Rate: str, DR: str, num_unsplit_alleles: str, GT: str, GQ: str, PL: str, AD: str, DP: str, AB: str, pAB: str, F_GT: str, F_GQ: str, F_DP: str, M_GT: str, M_GQ: str, M_DP: str, AC: str, AN: str, AF: str, gnomad_non_neuro_AF: str, gnomad_non_neuro_AF_popmax: str, Prior: str, P_De_Novo: str, Confidence: str, Category: str, Dataset: str}> fields. Counts by type:
  str: 61


In [15]:
df_asd = ht_asd.to_pandas()

In [6]:
# already wrote control ht
asd_no_controls = hl.read_table('gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/asd-lof-no-controls.ht')

In [7]:
asd_no_controls.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'is_pos_asd': bool 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [13]:
asd_no_controls.count()

2671

## CHD

In [8]:
ht_chd = hl.import_table(
    [
        'gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv',
        'gs://genetics-gym/raw_data/NIHMS906719-chd-dnm-controls.csv'
    ],
    source_file_field='case-control',
    delimiter = ','
)

2025-04-01 18:46:25.592 Hail: INFO: Reading table without type imputation1) / 1]
  Loading field 'Blinded ID' as type str (not specified)
  Loading field 'CHROM' as type str (not specified)
  Loading field 'POS' as type str (not specified)
  Loading field 'REF' as type str (not specified)
  Loading field 'ALT' as type str (not specified)
  Loading field 'Ensemble_GENEID' as type str (not specified)
  Loading field 'Gene' as type str (not specified)
  Loading field 'Variant_Class' as type str (not specified)
  Loading field 'AA_change' as type str (not specified)
  Loading field 'RadialSVM_score' as type str (not specified)
  Loading field 'RadialSVM_pred' as type str (not specified)
  Loading field 'ExAC_Overall' as type str (not specified)
  Loading field 'HHE Rank' as type str (not specified)
  Loading field 'pLI Score' as type str (not specified)


In [9]:
ht_chd.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Blinded ID': str 
    'CHROM': str 
    'POS': str 
    'REF': str 
    'ALT': str 
    'Ensemble_GENEID': str 
    'Gene': str 
    'Variant_Class': str 
    'AA_change': str 
    'RadialSVM_score': str 
    'RadialSVM_pred': str 
    'ExAC_Overall': str 
    'HHE Rank': str 
    'pLI Score': str 
    'case-control': str 
----------------------------------------
Key: []
----------------------------------------


In [10]:
ht_chd.show()

,,,,,,,,,,,,,,
Blinded ID,CHROM,POS,REF,ALT,Ensemble_GENEID,Gene,Variant_Class,AA_change,RadialSVM_score,RadialSVM_pred,ExAC_Overall,HHE Rank,pLI Score,case-control
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""GT04011952""","""19""","""8491545""","""C""","""G""","""ENSG00000099785""","""MARCH2""","""mis""","""p.L77V""","""-0.91""","""T""","""0.0001""","""70.2""","""0.25""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-00788""","""10""","""94070938""","""G""","""A""","""ENSG00000198060""","""MARCH5""","""mis""","""p.E28K""","""-1.05""","""T""",""".""","""75.1""","""0.9""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-01469""","""2""","""242275458""","""C""","""T""","""ENSG00000168385""","""SEPT2""","""mis""","""p.R106C""","""-0.131""","""T""",""".""","""98.1""","""0.87""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-06549""","""22""","""42382110""","""G""","""A""","""ENSG00000100167""","""SEPT3""","""mis""","""p.G109R""","""-0.936""","""T""",""".""","""50.4""","""0.07""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-05720""","""17""","""56604065""","""A""","""C""","""ENSG00000108387""","""SEPT4""","""mis""","""p.L127R""","""-0.936""","""T""",""".""","""54.4""","""0.01""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-07417""","""5""","""132097207""","""C""","""G""","""ENSG00000164402""","""SEPT8""","""misD""","""p.R302P""","""0.657""","""D""",""".""","""86.4""","""0.95""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-02674""","""12""","""53701325""","""G""","""C""","""ENSG00000094914""","""AAAS""","""mis""","""p.P530R""","""-0.358""","""T""",""".""","""80.1""","""0.0""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""GT04016012""","""12""","""125621409""","""C""","""T""","""ENSG00000081760""","""AACS""","""mis""","""p.P627L""","""-0.519""","""T""","""0.00001667""","""45.8""","""0.0""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""


### column counts

In [11]:
df_chd = ht_chd.to_pandas()
df_chd

,Blinded ID,CHROM,POS,REF,ALT,Ensemble_GENEID,Gene,Variant_Class,AA_change,RadialSVM_score,RadialSVM_pred,ExAC_Overall,HHE Rank,pLI Score,case-control
0,GT04011952,19,8491545,C,G,ENSG00000099785,MARCH2,mis,p.L77V,-0.91,T,0.0001,70.2,0.25,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
1,1-00788,10,94070938,G,A,ENSG00000198060,MARCH5,mis,p.E28K,-1.05,T,.,75.1,0.9,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
2,1-01469,2,242275458,C,T,ENSG00000168385,SEPT2,mis,p.R106C,-0.131,T,.,98.1,0.87,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
3,1-06549,22,42382110,G,A,ENSG00000100167,SEPT3,mis,p.G109R,-0.936,T,.,50.4,0.07,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
4,1-05720,17,56604065,A,C,ENSG00000108387,SEPT4,mis,p.L127R,-0.936,T,.,54.4,0.01,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4815,14018,11,60637182,TCCTC,T,ENSG00000149506,ZP1,frameshift,p.F164fs,.,.,.,17.8,0.00,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...
4816,14563,19,13915878,G,A,ENSG00000132003,ZSWIM4,mis,p.A210T,-1.124,T,8.40E-06,60.0,0.17,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...
4817,SSC07834,5,60768572,C,T,ENSG00000130449,ZSWIM6,syn,p.N247N,.,.,.,56.2,0.33,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...
4818,11474,11,113631634,G,A,ENSG00000086827,ZW10,mis,p.R83W,-0.843,T,6.63E-05,60.8,0.05,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...


In [12]:
df_chd['Variant_Class'].value_counts()

Variant_Class
mis           2436
syn           1185
misD           675
frameshift     245
non            194
splice          72
startloss        7
stoploss         6
Name: count, dtype: Int64

In [21]:
df_filt_chd = df_chd[df_chd['Variant_Class'].isin(['frameshift', 'non', 'splice'])]
df_filt_chd

,Blinded ID,CHROM,POS,REF,ALT,Ensemble_GENEID,Gene,Variant_Class,AA_change,RadialSVM_score,RadialSVM_pred,ExAC_Overall,HHE Rank,pLI Score,case-control
20,1-06692,17,48762187,C,T,ENSG00000108846,ABCC3,non,p.Q1411X,.,.,.,32.8,0.0,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
25,1-02675,10,27066140,G,A,ENSG00000136754,ABI1,non,p.R123X,.,.,8.53E-06,82.5,0.63,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
35,1-03806,1,226374124,C,A,ENSG00000182827,ACBD3,non,p.E85X,.,.,.,78.4,0.98,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
40,1-08490,7,100491441,C,CG,ENSG00000087085,ACHE,frameshift,p.R138fs,.,.,.,36.7,1.0,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
44,1-03160,5,131326601,G,T,ENSG00000164398,ACSL6,non,p.Y110X,.,.,.,26.0,0.09,gs://genetics-gym/temp/NIHMS906719-chd-dnm-cas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4749,SSC05284,5,840609,C,A,ENSG00000188818,ZDHHC11,splice,c.784+1G>T,-0.971,T,6.64E-05,11.7,0.00,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...
4770,11197,19,44513286,AT,A,ENSG00000159882,ZNF230,frameshift,p.M65fs,.,.,.,N/A,N/A,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...
4779,11003,5,178139609,G,A,ENSG00000169131,ZNF354A,non,p.R424X,.,.,1.65E-05,31.8,0.00,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...
4782,13842,18,72632469,G,A,ENSG00000215421,ZNF407,splice,c.5250-1G>A,.,.,.,48.1,1.00,gs://genetics-gym/raw_data/NIHMS906719-chd-dnm...


In [23]:
ht_filt_chd = hl.Table.from_pandas(df_filt_chd)

In [24]:
ht_filt_chd.show()

,,,,,,,,,,,,,,
Blinded ID,CHROM,POS,REF,ALT,Ensemble_GENEID,Gene,Variant_Class,AA_change,RadialSVM_score,RadialSVM_pred,ExAC_Overall,HHE Rank,pLI Score,case-control
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1-06692""","""17""","""48762187""","""C""","""T""","""ENSG00000108846""","""ABCC3""","""non""","""p.Q1411X""",""".""",""".""",""".""","""32.8""","""0.0""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-02675""","""10""","""27066140""","""G""","""A""","""ENSG00000136754""","""ABI1""","""non""","""p.R123X""",""".""",""".""","""8.53E-06""","""82.5""","""0.63""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-03806""","""1""","""226374124""","""C""","""A""","""ENSG00000182827""","""ACBD3""","""non""","""p.E85X""",""".""",""".""",""".""","""78.4""","""0.98""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-08490""","""7""","""100491441""","""C""","""CG""","""ENSG00000087085""","""ACHE""","""frameshift""","""p.R138fs""",""".""",""".""",""".""","""36.7""","""1.0""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-03160""","""5""","""131326601""","""G""","""T""","""ENSG00000164398""","""ACSL6""","""non""","""p.Y110X""",""".""",""".""",""".""","""26.0""","""0.09""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-06528""","""7""","""5568826""","""AG""","""A""","""ENSG00000075624""","""ACTB""","""frameshift""","""p.L110X""",""".""",""".""",""".""","""99.9""","""0.94""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-02279""","""11""","""62301129""","""C""","""A""","""ENSG00000124942""","""AHNAK""","""non""","""p.G254X""",""".""",""".""",""".""","""95.7""","""0.34""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""
"""1-01627""","""X""","""129263965""","""G""","""A""","""ENSG00000156709""","""AIFM1""","""non""","""p.R580X""",""".""",""".""",""".""","""83.6""","""0.98""","""gs://genetics-gym/temp/NIHMS906719-chd-dnm-cases-processed.csv"""


In [27]:
ht_filt_chd = ht_filt_chd.annotate(locus = hl.locus(ht_filt_chd.CHROM, hl.int(ht_filt_chd.POS), reference_genome='GRCh37'))
ht_filt_chd = ht_filt_chd.annotate(alleles = [ht_filt_chd.REF,ht_filt_chd.ALT])
ht_filt_chd = liftover_37_38(ht_filt_chd)
ht_filt_chd = ht_filt_chd.key_by('locus','alleles')
ht_filt_chd = ht_filt_chd.annotate(is_pos_chd = hl.if_else(ht_filt_chd['case-control'].contains('case'), True, False))
ht_filt_chd = ht_filt_chd.select('is_pos_chd')

In [28]:
ht_filt_chd.write('gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/chd-lof-no-controls.ht', overwrite=True)

2025-04-01 19:06:35.170 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-04-01 19:06:39.497 Hail: INFO: wrote table with 507 rows in 4 partitions to gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/chd-lof-no-controls.ht


In [29]:
ht_filt_chd.count()

507

## Combine Controls across datasets

In [32]:
ht_dd = dd_no_controls_1
ht_asd = asd_no_controls
ht_chd = ht_filt_chd

In [35]:
# Get controls from ASD set (is_pos_asd = False) and add them to DD set (is_pos_dd = False)
ht_dd_controls_from_asd = ht_asd.filter(~ht_asd.is_pos_asd)
ht_dd_controls_from_asd = ht_dd_controls_from_asd.annotate(is_pos_dd=False)
ht_dd_controls_from_asd = ht_dd_controls_from_asd.select('is_pos_dd')
ht_dd = ht_dd.union(ht_dd_controls_from_asd)

# Get controls from CHD set (is_pos_chd = False) and add them to DD set (is_pos_dd = False)
ht_dd_controls_from_chd = ht_chd.filter(~ht_chd.is_pos_chd)
ht_dd_controls_from_chd = ht_dd_controls_from_chd.annotate(is_pos_dd=False)
ht_dd_controls_from_chd = ht_dd_controls_from_chd.select('is_pos_dd')
ht_dd = ht_dd.union(ht_dd_controls_from_chd)

# asd and chd missing controls from dd?
ht_asd_controls_from_chd = ht_dd_controls_from_chd.rename({'is_pos_dd':'is_pos_asd'})
ht_asd = ht_asd.union(ht_asd_controls_from_chd)

ht_chd_controls_from_asd = ht_dd_controls_from_asd.rename({'is_pos_dd':'is_pos_chd'})
ht_chd = ht_chd.union(ht_chd_controls_from_asd)

ht_asd.write('gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/asd-lof-with-combined-controls.ht', overwrite=True)
ht_dd.write('gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/dd-lof-with-combined-controls.ht', overwrite=True)
ht_chd.write('gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/chd-lof-with-combined-controls.ht', overwrite=True)

2025-04-01 19:19:08.151 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-04-01 19:19:12.237 Hail: INFO: wrote table with 2815 rows in 5 partitions to gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/asd-lof-with-combined-controls.ht
2025-04-01 19:19:13.659 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-04-01 19:19:17.542 Hail: INFO: wrote table with 6523 rows in 6 partitions to gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/dd-lof-with-combined-controls.ht
2025-04-01 19:19:18.588 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-04-01 19:19:22.173 Hail: INFO: wrote table with 1005 rows in 5 partitions to gs://genetics-gym-not-public/Trisha/GSM_tables/eval_data/chd-lof-with-combined-controls.ht


In [38]:
ht_dd.count()

6523

In [34]:
# after filtering...
#ht_dd_controls_from_asd.show() 

,,
locus,alleles,is_pos_asd
locus<GRCh38>,array<str>,bool
chr1:943343,"[""T"",""TG""]",False
chr1:6593622,"[""TCC"",""T""]",False
chr1:7962762,"[""G"",""T""]",False
chr1:11957940,"[""TG"",""T""]",False
chr1:15767556,"[""CACCACAGGTACTGCCTG"",""C""]",False
chr1:20744881,"[""CTTCT"",""C""]",False
chr1:37552748,"[""C"",""CGGCTAACTCCTCGGGCTCGGTTCCCGCGG""]",False
chr1:41048837,"[""TC"",""T""]",False


# RVAS